In [1]:
import torch
import torch.nn as nn
from torchvision import models
from tools import  read_data
import pandas as pd
from torch.optim.lr_scheduler import StepLR
from kaggleImageFolder import KaggleImageFolder

In [2]:
batch_size = 512
labels_csv = './labels.csv'
root = 'D:/DATA/dog bread'


In [3]:
train_features = torch.load(root + '/train_features.pth')
valid_features = torch.load(root + '/valid_features.pth')
test_features = torch.load(root + '/test_features.pth')

In [4]:
def accuracy(model, dataloader, n_data):
    correct = 0
    model.eval()
    for x, y in dataloader:
        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()
        z = model(x)
        yhat = torch.argmax(z, 1)
        correct += (y == yhat).sum().item()

    acc = correct / n_data
    return acc

In [5]:
class Classifier(nn.Module):
    def __init__(self, input_dim, n_classes):
        super(Classifier, self).__init__()
        self.arch = self.__class__.__name__
        self.n_classes = n_classes
        
        self.model = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.Dropout(p=0.5),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 256),
            nn.Dropout(p=0.5),
            nn.ReLU(inplace=True),
            nn.Linear(256, n_classes)
        )
        torch.nn.init.kaiming_uniform_(self.model[0].weight, nonlinearity='relu')
        torch.nn.init.kaiming_uniform_(self.model[3].weight, nonlinearity='relu')
        torch.nn.init.kaiming_uniform_(self.model[-1].weight, nonlinearity='relu')
        
    def forward(self, x):
        x = self.model(x)
        return x
    
    def save_model(self, save_path):
        checkpoint = {
            'model_state_dict': self.state_dict(),
            'n_classes': self.n_classes
        }
        torch.save(checkpoint, save_path)

In [6]:
def train(model, train_dataloader, valid_dataloader, n_train, n_val, criterion, optimizer, epochs=20, scheduler=None, save_path='./'):
    train_loss_list = []
    train_acc_list = []
    valid_acc_list = []

    it = 1

    for epoch in range(epochs):
        if scheduler is not None:
            scheduler.step()

        epoch_losses = []
        model.train()
        for i, (x, y) in enumerate(train_dataloader):
            if torch.cuda.is_available():
                x = x.cuda()
                y = y.cuda()
            optimizer.zero_grad()
            z = model(x)
            loss = criterion(z, y)
            epoch_losses.append(loss.data.item())
            loss.backward()
            optimizer.step()

            it += 1

            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        mean_epoch_loss = sum(epoch_losses) / len(epoch_losses)
        print("Epoch: {}/{}...".format(epoch + 1, epochs),
              "Train Loss: {:.4f}".format(mean_epoch_loss))
        train_loss_list.append(mean_epoch_loss)

        train_acc_list.append(accuracy(model, train_dataloader, n_train))

        if valid_dataloader is not None:
            acc = accuracy(model, valid_dataloader, n_val)
            if len(valid_acc_list) > 0 and acc > max(valid_acc_list):
                model.save_model(save_path + '/{}-checkpoint-best.pth'.format(model.arch))
                optimizer_params = {"optimizer_params": optimizer.state_dict()}
                torch.save(optimizer_params, save_path + "/{}-{}-checkpoint-best.pth".format(
                    model.arch, optimizer.__class__.__name__))
            valid_acc_list.append(acc)

            print("Epoch: {}/{}... ".format(epoch + 1, epochs),
                  "Iter: {}".format(it),
                  "valid accuracy: {:.4f}".format(valid_acc_list[-1]))
            print("Epoch: {}/{}... ".format(epoch + 1, epochs),
                  "Iter: {}".format(it),
                  "train accuracy: {:.4f}".format(train_acc_list[-1]))

    return train_loss_list, train_acc_list, valid_acc_list

In [7]:
import torch.nn.functional as F
import os
def predict(test_data, model_file, sample_submission, input_dim, arch):
    submission_samples = pd.read_csv(sample_submission)
    submission_samples = submission_samples.set_index('id')

    data_len = len(test_data)
    print('test data len:', data_len)
    
    test_images = KaggleImageFolder('d:/DATA/dog bread/test', transform=None, mode='test').samples
    
    test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=1)
    
    checkpoint = torch.load(model_file)
    n_classes = checkpoint['n_classes']
    model = Classifier(input_dim, n_classes)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    cuda = torch.cuda.is_available()
    if cuda:
        model.cuda()

    with torch.no_grad():
        for i, (image, x) in enumerate(zip(test_images, test_loader)):
            if cuda:
                x = x.cuda()
                torch.backends.cudnn.benchmark = True
            z = model(x)
            prob = F.softmax(z, dim=1).cpu()[0].numpy().tolist()
            image = os.path.basename(image).split('.')[0]

            submission_samples.loc[image] = prob

    submission_samples.to_csv('sample_submission_{}.csv'.format(arch))

### restnet152 0.40934

In [8]:
train_dataloader = torch.utils.data.DataLoader(dataset=train_features['resnet152'], batch_size=batch_size, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(dataset=valid_features['resnet152'], batch_size=batch_size)
model = Classifier(train_features['resnet152'][0][0].size()[0], 120)
optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, model.parameters()),
                                 lr=1e-4, weight_decay=1e-5)

criterion = torch.nn.CrossEntropyLoss()

if torch.cuda.is_available():
    model.cuda()
    criterion.cuda()
    torch.backends.cudnn.benchmark = True

In [9]:
# scheduler = StepLR(optimizer, step_size=100, gamma=0.1)
train_loss_list, train_acc_list, valid_acc_list = train(
    model, train_dataloader, valid_dataloader, n_train=len(train_features['resnet152']), n_val=len(valid_features['resnet152']), 
    criterion=criterion, optimizer=optimizer, epochs=100, save_path=root+'/save_models', scheduler=None
)
model.save_model(root+'/save_models/resnet152.pth')

Epoch: 1/100... Train Loss: 5.1472
Epoch: 1/100...  Iter: 18 valid accuracy: 0.0780
Epoch: 1/100...  Iter: 18 train accuracy: 0.0640
Epoch: 2/100... Train Loss: 4.7110
Epoch: 2/100...  Iter: 35 valid accuracy: 0.1539
Epoch: 2/100...  Iter: 35 train accuracy: 0.1477
Epoch: 3/100... Train Loss: 4.5348
Epoch: 3/100...  Iter: 52 valid accuracy: 0.2772
Epoch: 3/100...  Iter: 52 train accuracy: 0.2767
Epoch: 4/100... Train Loss: 4.3186
Epoch: 4/100...  Iter: 69 valid accuracy: 0.3961
Epoch: 4/100...  Iter: 69 train accuracy: 0.4064
Epoch: 5/100... Train Loss: 4.0608
Epoch: 5/100...  Iter: 86 valid accuracy: 0.5048
Epoch: 5/100...  Iter: 86 train accuracy: 0.5163
Epoch: 6/100... Train Loss: 3.7704
Epoch: 6/100...  Iter: 103 valid accuracy: 0.5813
Epoch: 6/100...  Iter: 103 train accuracy: 0.5913
Epoch: 7/100... Train Loss: 3.4377
Epoch: 7/100...  Iter: 120 valid accuracy: 0.6464
Epoch: 7/100...  Iter: 120 train accuracy: 0.6621
Epoch: 8/100... Train Loss: 3.1391
Epoch: 8/100...  Iter: 137 val

Epoch: 61/100... Train Loss: 0.5541
Epoch: 61/100...  Iter: 1038 valid accuracy: 0.9225
Epoch: 61/100...  Iter: 1038 train accuracy: 0.9483
Epoch: 62/100... Train Loss: 0.5409
Epoch: 62/100...  Iter: 1055 valid accuracy: 0.9225
Epoch: 62/100...  Iter: 1055 train accuracy: 0.9514
Epoch: 63/100... Train Loss: 0.5539
Epoch: 63/100...  Iter: 1072 valid accuracy: 0.9247
Epoch: 63/100...  Iter: 1072 train accuracy: 0.9515
Epoch: 64/100... Train Loss: 0.5248
Epoch: 64/100...  Iter: 1089 valid accuracy: 0.9263
Epoch: 64/100...  Iter: 1089 train accuracy: 0.9533
Epoch: 65/100... Train Loss: 0.5226
Epoch: 65/100...  Iter: 1106 valid accuracy: 0.9247
Epoch: 65/100...  Iter: 1106 train accuracy: 0.9527
Epoch: 66/100... Train Loss: 0.5124
Epoch: 66/100...  Iter: 1123 valid accuracy: 0.9257
Epoch: 66/100...  Iter: 1123 train accuracy: 0.9540
Epoch: 67/100... Train Loss: 0.5111
Epoch: 67/100...  Iter: 1140 valid accuracy: 0.9241
Epoch: 67/100...  Iter: 1140 train accuracy: 0.9543
Epoch: 68/100... Tra

In [10]:
model_file = u"d:/DATA/dog bread/save_models/Classifier-checkpoint-best.pth"
sample_submission = 'sample_submission.csv'
predict(test_features['resnet152'], model_file,sample_submission,input_dim=2048, arch='resnet152' )

test data len: 10357


### denset169 0.66725

In [11]:
train_dataloader = torch.utils.data.DataLoader(dataset=train_features['densenet169'], batch_size=batch_size, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(dataset=valid_features['densenet169'], batch_size=batch_size)
model = Classifier(train_features['densenet169'][0][0].size()[0], 120)
optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, model.parameters()),
                                 lr=1e-3, weight_decay=1e-5)

criterion = torch.nn.CrossEntropyLoss()

if torch.cuda.is_available():
    model.cuda()
    criterion.cuda()
    torch.backends.cudnn.benchmark = True

In [12]:
train_loss_list, train_acc_list, valid_acc_list = train(
    model, train_dataloader, valid_dataloader, n_train=len(train_features['densenet169']), n_val=len(valid_features['densenet169']), 
    criterion=criterion, optimizer=optimizer, epochs=50, save_path=root+'/save_models', scheduler=None
)
model.save_model(root+'/save_models/densenet169.pth')

Epoch: 1/50... Train Loss: 4.9022
Epoch: 1/50...  Iter: 18 valid accuracy: 0.1297
Epoch: 1/50...  Iter: 18 train accuracy: 0.1328
Epoch: 2/50... Train Loss: 4.1944
Epoch: 2/50...  Iter: 35 valid accuracy: 0.4483
Epoch: 2/50...  Iter: 35 train accuracy: 0.4491
Epoch: 3/50... Train Loss: 3.2114
Epoch: 3/50...  Iter: 52 valid accuracy: 0.6609
Epoch: 3/50...  Iter: 52 train accuracy: 0.6587
Epoch: 4/50... Train Loss: 2.3343
Epoch: 4/50...  Iter: 69 valid accuracy: 0.7487
Epoch: 4/50...  Iter: 69 train accuracy: 0.7590
Epoch: 5/50... Train Loss: 1.8100
Epoch: 5/50...  Iter: 86 valid accuracy: 0.7917
Epoch: 5/50...  Iter: 86 train accuracy: 0.8111
Epoch: 6/50... Train Loss: 1.4899
Epoch: 6/50...  Iter: 103 valid accuracy: 0.8165
Epoch: 6/50...  Iter: 103 train accuracy: 0.8343
Epoch: 7/50... Train Loss: 1.3098
Epoch: 7/50...  Iter: 120 valid accuracy: 0.8358
Epoch: 7/50...  Iter: 120 train accuracy: 0.8553
Epoch: 8/50... Train Loss: 1.1636
Epoch: 8/50...  Iter: 137 valid accuracy: 0.8520
Epo

In [13]:
model_file = u"d:/DATA/dog bread/save_models/densenet169.pth"
sample_submission = 'sample_submission.csv'
predict(test_features['densenet169'], model_file,sample_submission,input_dim=train_features['densenet169'][0][0].size()[0], arch='densenet169')

test data len: 10357


### inception_v3  3.32490

In [14]:
train_dataloader = torch.utils.data.DataLoader(dataset=train_features['inception_v3'], batch_size=batch_size, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(dataset=valid_features['inception_v3'], batch_size=batch_size)
model = Classifier(train_features['inception_v3'][0][0].size()[0], 120)
optimizer = torch.optim.SGD(params=filter(lambda p: p.requires_grad, model.parameters()),
                                 lr=1e-1, weight_decay=1e-5)

criterion = torch.nn.CrossEntropyLoss()

if torch.cuda.is_available():
    model.cuda()
    criterion.cuda()
    torch.backends.cudnn.benchmark = True

In [15]:
scheduler = StepLR(optimizer, step_size=300, gamma=0.1)
train_loss_list, train_acc_list, valid_acc_list = train(
    model, train_dataloader, valid_dataloader, n_train=len(train_features['inception_v3']), n_val=len(valid_features['inception_v3']), 
    criterion=criterion, optimizer=optimizer, epochs=500, save_path=root+'/save_models', scheduler=scheduler
)
model.save_model(root+'/save_models/inception_v3.pth')

Epoch: 1/500... Train Loss: 4.8668
Epoch: 1/500...  Iter: 18 valid accuracy: 0.0112
Epoch: 1/500...  Iter: 18 train accuracy: 0.0134
Epoch: 2/500... Train Loss: 4.7980
Epoch: 2/500...  Iter: 35 valid accuracy: 0.0145
Epoch: 2/500...  Iter: 35 train accuracy: 0.0141
Epoch: 3/500... Train Loss: 4.7907
Epoch: 3/500...  Iter: 52 valid accuracy: 0.0139
Epoch: 3/500...  Iter: 52 train accuracy: 0.0165
Epoch: 4/500... Train Loss: 4.7876
Epoch: 4/500...  Iter: 69 valid accuracy: 0.0166
Epoch: 4/500...  Iter: 69 train accuracy: 0.0164
Epoch: 5/500... Train Loss: 4.7764
Epoch: 5/500...  Iter: 86 valid accuracy: 0.0171
Epoch: 5/500...  Iter: 86 train accuracy: 0.0148
Epoch: 6/500... Train Loss: 4.7714
Epoch: 6/500...  Iter: 103 valid accuracy: 0.0177
Epoch: 6/500...  Iter: 103 train accuracy: 0.0190
Epoch: 7/500... Train Loss: 4.7680
Epoch: 7/500...  Iter: 120 valid accuracy: 0.0273
Epoch: 7/500...  Iter: 120 train accuracy: 0.0225
Epoch: 8/500... Train Loss: 4.7623
Epoch: 8/500...  Iter: 137 val

Epoch: 61/500... Train Loss: 3.7248
Epoch: 61/500...  Iter: 1038 valid accuracy: 0.2082
Epoch: 61/500...  Iter: 1038 train accuracy: 0.2370
Epoch: 62/500... Train Loss: 3.6867
Epoch: 62/500...  Iter: 1055 valid accuracy: 0.2131
Epoch: 62/500...  Iter: 1055 train accuracy: 0.2390
Epoch: 63/500... Train Loss: 3.6932
Epoch: 63/500...  Iter: 1072 valid accuracy: 0.1638
Epoch: 63/500...  Iter: 1072 train accuracy: 0.1731
Epoch: 64/500... Train Loss: 3.6663
Epoch: 64/500...  Iter: 1089 valid accuracy: 0.1692
Epoch: 64/500...  Iter: 1089 train accuracy: 0.1844
Epoch: 65/500... Train Loss: 3.6434
Epoch: 65/500...  Iter: 1106 valid accuracy: 0.2136
Epoch: 65/500...  Iter: 1106 train accuracy: 0.2323
Epoch: 66/500... Train Loss: 3.6094
Epoch: 66/500...  Iter: 1123 valid accuracy: 0.1815
Epoch: 66/500...  Iter: 1123 train accuracy: 0.2051
Epoch: 67/500... Train Loss: 3.6269
Epoch: 67/500...  Iter: 1140 valid accuracy: 0.2002
Epoch: 67/500...  Iter: 1140 train accuracy: 0.2151
Epoch: 68/500... Tra

Epoch: 120/500... Train Loss: 3.0092
Epoch: 120/500...  Iter: 2041 valid accuracy: 0.2950
Epoch: 120/500...  Iter: 2041 train accuracy: 0.3321
Epoch: 121/500... Train Loss: 2.9864
Epoch: 121/500...  Iter: 2058 valid accuracy: 0.3389
Epoch: 121/500...  Iter: 2058 train accuracy: 0.3865
Epoch: 122/500... Train Loss: 2.9750
Epoch: 122/500...  Iter: 2075 valid accuracy: 0.3207
Epoch: 122/500...  Iter: 2075 train accuracy: 0.3687
Epoch: 123/500... Train Loss: 2.9448
Epoch: 123/500...  Iter: 2092 valid accuracy: 0.3314
Epoch: 123/500...  Iter: 2092 train accuracy: 0.3811
Epoch: 124/500... Train Loss: 2.9290
Epoch: 124/500...  Iter: 2109 valid accuracy: 0.2912
Epoch: 124/500...  Iter: 2109 train accuracy: 0.3327
Epoch: 125/500... Train Loss: 2.9404
Epoch: 125/500...  Iter: 2126 valid accuracy: 0.2875
Epoch: 125/500...  Iter: 2126 train accuracy: 0.3250
Epoch: 126/500... Train Loss: 2.9332
Epoch: 126/500...  Iter: 2143 valid accuracy: 0.2778
Epoch: 126/500...  Iter: 2143 train accuracy: 0.3388

Epoch: 178/500... Train Loss: 2.5484
Epoch: 178/500...  Iter: 3027 valid accuracy: 0.3833
Epoch: 178/500...  Iter: 3027 train accuracy: 0.4341
Epoch: 179/500... Train Loss: 2.5309
Epoch: 179/500...  Iter: 3044 valid accuracy: 0.3581
Epoch: 179/500...  Iter: 3044 train accuracy: 0.4354
Epoch: 180/500... Train Loss: 2.5132
Epoch: 180/500...  Iter: 3061 valid accuracy: 0.4101
Epoch: 180/500...  Iter: 3061 train accuracy: 0.4822
Epoch: 181/500... Train Loss: 2.5209
Epoch: 181/500...  Iter: 3078 valid accuracy: 0.3260
Epoch: 181/500...  Iter: 3078 train accuracy: 0.3703
Epoch: 182/500... Train Loss: 2.5078
Epoch: 182/500...  Iter: 3095 valid accuracy: 0.3522
Epoch: 182/500...  Iter: 3095 train accuracy: 0.4401
Epoch: 183/500... Train Loss: 2.5491
Epoch: 183/500...  Iter: 3112 valid accuracy: 0.4117
Epoch: 183/500...  Iter: 3112 train accuracy: 0.4894
Epoch: 184/500... Train Loss: 2.5454
Epoch: 184/500...  Iter: 3129 valid accuracy: 0.3838
Epoch: 184/500...  Iter: 3129 train accuracy: 0.4835

Epoch: 236/500... Train Loss: 2.2383
Epoch: 236/500...  Iter: 4013 valid accuracy: 0.3988
Epoch: 236/500...  Iter: 4013 train accuracy: 0.5139
Epoch: 237/500... Train Loss: 2.2398
Epoch: 237/500...  Iter: 4030 valid accuracy: 0.4154
Epoch: 237/500...  Iter: 4030 train accuracy: 0.5067
Epoch: 238/500... Train Loss: 2.2223
Epoch: 238/500...  Iter: 4047 valid accuracy: 0.4390
Epoch: 238/500...  Iter: 4047 train accuracy: 0.5592
Epoch: 239/500... Train Loss: 2.1828
Epoch: 239/500...  Iter: 4064 valid accuracy: 0.4475
Epoch: 239/500...  Iter: 4064 train accuracy: 0.5568
Epoch: 240/500... Train Loss: 2.1634
Epoch: 240/500...  Iter: 4081 valid accuracy: 0.4010
Epoch: 240/500...  Iter: 4081 train accuracy: 0.4951
Epoch: 241/500... Train Loss: 2.2518
Epoch: 241/500...  Iter: 4098 valid accuracy: 0.4352
Epoch: 241/500...  Iter: 4098 train accuracy: 0.5390
Epoch: 242/500... Train Loss: 2.1632
Epoch: 242/500...  Iter: 4115 valid accuracy: 0.4010
Epoch: 242/500...  Iter: 4115 train accuracy: 0.5030

Epoch: 294/500... Train Loss: 1.9939
Epoch: 294/500...  Iter: 4999 valid accuracy: 0.4524
Epoch: 294/500...  Iter: 4999 train accuracy: 0.6047
Epoch: 295/500... Train Loss: 1.9170
Epoch: 295/500...  Iter: 5016 valid accuracy: 0.4507
Epoch: 295/500...  Iter: 5016 train accuracy: 0.5949
Epoch: 296/500... Train Loss: 2.0000
Epoch: 296/500...  Iter: 5033 valid accuracy: 0.4609
Epoch: 296/500...  Iter: 5033 train accuracy: 0.5889
Epoch: 297/500... Train Loss: 1.9725
Epoch: 297/500...  Iter: 5050 valid accuracy: 0.4438
Epoch: 297/500...  Iter: 5050 train accuracy: 0.5658
Epoch: 298/500... Train Loss: 1.9686
Epoch: 298/500...  Iter: 5067 valid accuracy: 0.4309
Epoch: 298/500...  Iter: 5067 train accuracy: 0.5635
Epoch: 299/500... Train Loss: 1.9557
Epoch: 299/500...  Iter: 5084 valid accuracy: 0.4416
Epoch: 299/500...  Iter: 5084 train accuracy: 0.5865
Epoch: 300/500... Train Loss: 2.0159
Epoch: 300/500...  Iter: 5101 valid accuracy: 0.4507
Epoch: 300/500...  Iter: 5101 train accuracy: 0.5933

Epoch: 352/500... Train Loss: 1.5909
Epoch: 352/500...  Iter: 5985 valid accuracy: 0.5375
Epoch: 352/500...  Iter: 5985 train accuracy: 0.7398
Epoch: 353/500... Train Loss: 1.5870
Epoch: 353/500...  Iter: 6002 valid accuracy: 0.5396
Epoch: 353/500...  Iter: 6002 train accuracy: 0.7411
Epoch: 354/500... Train Loss: 1.5863
Epoch: 354/500...  Iter: 6019 valid accuracy: 0.5412
Epoch: 354/500...  Iter: 6019 train accuracy: 0.7421
Epoch: 355/500... Train Loss: 1.5757
Epoch: 355/500...  Iter: 6036 valid accuracy: 0.5418
Epoch: 355/500...  Iter: 6036 train accuracy: 0.7406
Epoch: 356/500... Train Loss: 1.5886
Epoch: 356/500...  Iter: 6053 valid accuracy: 0.5418
Epoch: 356/500...  Iter: 6053 train accuracy: 0.7411
Epoch: 357/500... Train Loss: 1.5770
Epoch: 357/500...  Iter: 6070 valid accuracy: 0.5418
Epoch: 357/500...  Iter: 6070 train accuracy: 0.7426
Epoch: 358/500... Train Loss: 1.5760
Epoch: 358/500...  Iter: 6087 valid accuracy: 0.5385
Epoch: 358/500...  Iter: 6087 train accuracy: 0.7407

Epoch: 410/500... Train Loss: 1.5206
Epoch: 410/500...  Iter: 6971 valid accuracy: 0.5455
Epoch: 410/500...  Iter: 6971 train accuracy: 0.7513
Epoch: 411/500... Train Loss: 1.5351
Epoch: 411/500...  Iter: 6988 valid accuracy: 0.5460
Epoch: 411/500...  Iter: 6988 train accuracy: 0.7549
Epoch: 412/500... Train Loss: 1.5477
Epoch: 412/500...  Iter: 7005 valid accuracy: 0.5407
Epoch: 412/500...  Iter: 7005 train accuracy: 0.7537
Epoch: 413/500... Train Loss: 1.5536
Epoch: 413/500...  Iter: 7022 valid accuracy: 0.5482
Epoch: 413/500...  Iter: 7022 train accuracy: 0.7543
Epoch: 414/500... Train Loss: 1.5416
Epoch: 414/500...  Iter: 7039 valid accuracy: 0.5450
Epoch: 414/500...  Iter: 7039 train accuracy: 0.7556
Epoch: 415/500... Train Loss: 1.5419
Epoch: 415/500...  Iter: 7056 valid accuracy: 0.5439
Epoch: 415/500...  Iter: 7056 train accuracy: 0.7550
Epoch: 416/500... Train Loss: 1.5491
Epoch: 416/500...  Iter: 7073 valid accuracy: 0.5471
Epoch: 416/500...  Iter: 7073 train accuracy: 0.7557

Epoch: 468/500... Train Loss: 1.4863
Epoch: 468/500...  Iter: 7957 valid accuracy: 0.5482
Epoch: 468/500...  Iter: 7957 train accuracy: 0.7652
Epoch: 469/500... Train Loss: 1.5088
Epoch: 469/500...  Iter: 7974 valid accuracy: 0.5546
Epoch: 469/500...  Iter: 7974 train accuracy: 0.7661
Epoch: 470/500... Train Loss: 1.5189
Epoch: 470/500...  Iter: 7991 valid accuracy: 0.5541
Epoch: 470/500...  Iter: 7991 train accuracy: 0.7665
Epoch: 471/500... Train Loss: 1.4976
Epoch: 471/500...  Iter: 8008 valid accuracy: 0.5546
Epoch: 471/500...  Iter: 8008 train accuracy: 0.7657
Epoch: 472/500... Train Loss: 1.5102
Epoch: 472/500...  Iter: 8025 valid accuracy: 0.5514
Epoch: 472/500...  Iter: 8025 train accuracy: 0.7684
Epoch: 473/500... Train Loss: 1.4946
Epoch: 473/500...  Iter: 8042 valid accuracy: 0.5503
Epoch: 473/500...  Iter: 8042 train accuracy: 0.7681
Epoch: 474/500... Train Loss: 1.4814
Epoch: 474/500...  Iter: 8059 valid accuracy: 0.5514
Epoch: 474/500...  Iter: 8059 train accuracy: 0.7688

In [16]:
model_file = u"d:/DATA/dog bread/save_models/inception_v3.pth"
sample_submission = 'sample_submission.csv'
predict(test_features['inception_v3'], model_file,sample_submission,input_dim=train_features['inception_v3'][0][0].size()[0], arch='inception_v3')

test data len: 10357


### cat 2 features 0.40715

In [17]:
def get_mix_features_labels(features):
    mix_features = []
    for x1, x2 in zip(features['resnet152'], features['densenet169']):
        assert x1[1] == x2[1], "labels did not match"
        mix_features.append((torch.cat((x1[0], x2[0]), 0), x1[1]))
    return mix_features

In [18]:
def get_mix_features(features):
    mix_features = []
    for x1, x2 in zip(features['resnet152'], features['densenet169']):
        mix_features.append(torch.cat((x1, x2,), 0))
    return mix_features

In [19]:
mix_trian_features = get_mix_features_labels(train_features)
mix_valid_features = get_mix_features_labels(valid_features)
mix_test_features = get_mix_features(test_features)

In [20]:
mix_trian_features[2][0].size()

torch.Size([3712])

In [21]:
mix_valid_features[0][0].size()

torch.Size([3712])

In [22]:
mix_test_features[0].size()

torch.Size([3712])

In [23]:
input_dim = mix_trian_features[0][0].size()[0]

In [24]:
train_dataloader = torch.utils.data.DataLoader(dataset=mix_trian_features, batch_size=batch_size, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(dataset=mix_valid_features, batch_size=batch_size)
model = Classifier(input_dim, 120)

criterion = torch.nn.CrossEntropyLoss()

if torch.cuda.is_available():
    model.cuda()
    criterion.cuda()
    torch.backends.cudnn.benchmark = True

In [30]:
optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, model.parameters()),
                                 lr=1e-4, weight_decay=1e-5)
train_loss_list, train_acc_list, valid_acc_list = train(
    model, train_dataloader, valid_dataloader, n_train=len(mix_trian_features), n_val=len(mix_valid_features), 
    criterion=criterion, optimizer=optimizer, epochs=20, save_path=root+'/save_models', scheduler=None
)

Epoch: 1/20... Train Loss: 0.1721
Epoch: 1/20...  Iter: 18 valid accuracy: 0.9682
Epoch: 1/20...  Iter: 18 train accuracy: 0.9973
Epoch: 2/20... Train Loss: 0.1734
Epoch: 2/20...  Iter: 35 valid accuracy: 0.9666
Epoch: 2/20...  Iter: 35 train accuracy: 0.9980
Epoch: 3/20... Train Loss: 0.1582
Epoch: 3/20...  Iter: 52 valid accuracy: 0.9688
Epoch: 3/20...  Iter: 52 train accuracy: 0.9987
Epoch: 4/20... Train Loss: 0.1579
Epoch: 4/20...  Iter: 69 valid accuracy: 0.9682
Epoch: 4/20...  Iter: 69 train accuracy: 0.9988
Epoch: 5/20... Train Loss: 0.1455
Epoch: 5/20...  Iter: 86 valid accuracy: 0.9688
Epoch: 5/20...  Iter: 86 train accuracy: 0.9986
Epoch: 6/20... Train Loss: 0.1473
Epoch: 6/20...  Iter: 103 valid accuracy: 0.9688
Epoch: 6/20...  Iter: 103 train accuracy: 0.9980
Epoch: 7/20... Train Loss: 0.1364
Epoch: 7/20...  Iter: 120 valid accuracy: 0.9672
Epoch: 7/20...  Iter: 120 train accuracy: 0.9989
Epoch: 8/20... Train Loss: 0.1363
Epoch: 8/20...  Iter: 137 valid accuracy: 0.9661
Epo

In [26]:
# optimizer = torch.optim.Adam(params=filter(lambda p: p.requires_grad, model.parameters()),
#                                  lr=1e-4, weight_decay=1e-5)
# scheduler = StepLR(optimizer, step_size=100, gamma=0.1)
# train_loss_list, train_acc_list, valid_acc_list = train(
#     model, train_dataloader, valid_dataloader, n_train=len(mix_trian_features), n_val=len(mix_valid_features), 
#     criterion=criterion, optimizer=optimizer, epochs=50, save_path=root+'/save_models', scheduler=scheduler
# )

In [27]:
model.save_model(root+'/save_models/mix-model.pth')

In [28]:
model_file = u"d:/DATA/dog bread/save_models/mix-model.pth"
sample_submission = 'sample_submission.csv'
predict(mix_test_features, model_file,sample_submission,input_dim=input_dim, arch='mix')

test data len: 10357
